**`Webinar 2024-02-05`**

[YouTube Recording](https://www.youtube.com/watch?v=W_d7sERdrgY&list=PLePKvgYhNOFxg3O7gXfmxaKp0EWXEhvNC&index=9)

# AIs Debating Science and Philosophy

##### String Theory vs. Loop Quantum Gravity
##### Stoicism vs. Epicureanism

### Flow of the debate

1. Run both AI chat bots' knowledge bases through a document review where the purpose is to generate questions in relation to each philosophy / concept.
2. Have a 3rd moderator chatbot come up with a set of simple questions to ask based on the questions generated in the previous document reviews.
3. Let boths AI chat bots have their opening statements.
4. Have the AIs both answer questions generated by the moderator.
5. Create 3 documents in a new KB specifically created for this debate:
    - The opening statement of the string theory AI
    - The opening statement of the loop quantum gravity AI
    - The full conversation
6. Run a document review of the debate KB as a review of the document
7. Use output from previous step as input to long-form endpoint to do one final review of tbe debate, the items discussed
8. Export long form essay to audio

##### `NOTE: To run this, you will need to replace with your own topic KBs, topic names, and descriptions.`

In [1]:
import superpowered
import requests
import time
import tqdm

from IPython.display import display, Markdown

SP_BASE_URL = 'https://api.superpowered.ai/v1'
SP_API_KEY_ID = ''
SP_API_KEY_SECRET = ''

API_AUTH = (SP_API_KEY_ID, SP_API_KEY_SECRET)
superpowered.set_api_key(SP_API_KEY_ID, SP_API_KEY_SECRET)

# SETTING TOPICS
TOPIC_1_KB_ID = '46f8be79-61dc-4516-b92f-c51b7e8f19e8'     # STRING THEORY
TOPIC_2_KB_ID = 'c3fb0f72-a42e-4b8f-a3ab-695f08503e58'     # LOOP QUANTUM GRAVITY
debate_description = 'Theory of Everything'
topic_1_name = 'String Theory'
topic_2_name = 'Loop Quantum Gravity'


# TOPIC_1_KB_ID = 'cf1b7a9b-eac2-4a27-a24d-dfc8e0d3aeb8'     # STOICISM
# TOPIC_2_KB_ID = 'd595f5e6-b81e-413c-87c1-b4382909d471'     # EPICUREANISM
# debate_description = 'Philolophy of Optimal Living'
# topic_1_name = 'Stoicism'
# topic_2_name = 'Epicureanism'


### 1. Document Review for each knowledge base

In [2]:
def run_review(kb_id: str, ar_instructions: str, fr_instructions: str):
    url = f'{SP_BASE_URL}/knowledge_bases/{kb_id}/review'
    payload = {
        'active_reading_instructions': ar_instructions,
        'final_review_instructions': fr_instructions,
        'reference_knowledge_base_ids': [TOPIC_1_KB_ID, TOPIC_2_KB_ID]
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    final_review_display = display(Markdown('Review pending...'), display_id=True)
    progress_bar = tqdm.tqdm(total=1)
    progress_bar.set_description('Active Reading Progress')

    # keep request job until it's completed (or failed)
    while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
        resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
        progress_bar.n = resp.json()['response']['active_reading_progress_pct']
        if progress_bar.n >= 1:
            final_review_display.update(Markdown(resp.json()["response"]["final_review"]))
        progress_bar.refresh()

    progress_bar.close()
    return resp.json()['response']['final_review']

In [3]:
active_reading_instructions = f"""\
You are reading through a knowledge base with information about a {debate_description}. 

Make note of information that are hard evidence as well as information that is speculative.
"""

final_review_instructions = """\
You are moderating a debate. Given all of the knowledge you have read, please generate a list of questions separated by a new line.

During your active reading, you made note of information that are hard evidence as well as information that is speculative. Use this information to generate questions that are relevant to the evidence and implications of the claims made in the knowledge base.

EXAMPLE OUTPUT:

What is the evidence for <insert claim here>?
What are the implications of <insert claim here>?

The questions you generate should be as intro-level as possible and should be relevant to the evidence and implications of the claims made in the knowledge base.
"""

# RUN KNOWLEDGE BASE REVIEWS
topic_1_review = run_review(TOPIC_1_KB_ID, active_reading_instructions, final_review_instructions)
topic_2_review = run_review(TOPIC_2_KB_ID, active_reading_instructions, final_review_instructions)

Based on the extracted information, here is a list of questions that are relevant to the evidence and implications of the claims made in the knowledge base:

1. How does string theory propose to replace point-like particles with one-dimensional strings?
2. In what ways has string theory contributed to advances in various fields of physics such as black hole physics and early universe cosmology?
3. Can you explain the significance of the AdS/CFT correspondence in string theory?
4. What are the implications of string theory potentially providing a unified description of gravity and particle physics?
5. How does the discovery of dualities between different versions of string theory impact our understanding of the theory?
6. What is the hard evidence supporting the existence of the BFSS matrix model and its equivalence to M-theory?
7. How did the work of Strominger and Vafa contribute to our understanding of black holes in string theory?
8. What are the speculative aspects of M-theory's true meaning and structure?
9. How does string theory address the black hole information paradox?
10. What is the connection between string theory and the classification theorem's sporadic groups, such as the monster group?
11. What are the challenges in defining the full theory of string theory in all circumstances?
12. How does string theory's requirement for extra dimensions of spacetime affect its mathematical consistency?
13. What are the speculative implications of string theory's large number of vacuum states for our understanding of the universe?
14. How does string theory's lack of manifest background independence impact its acceptance as a theory of everything?
15. What are the criticisms of string theory in terms of its impact on the future of fundamental physics?
16. How does loop quantum gravity differ from string theory in its approach to quantum gravity?
17. What are the potential observable cosmological signatures that string theory might leave, which could be detected by future technologies?
18. How does the speculative nature of string theory's relationship with experimental observations affect its status as a scientific theory?
19. What are the implications of the discovery of supersymmetry for string theory?
20. How does string theory aim to merge quantum mechanics with general relativity, and what evidence supports this aim?

These questions are designed to probe both the established aspects of string theory and the speculative elements that require further investigation or confirmation.

Active Reading Progress: 100%|██████████| 1/1 [02:27<00:00, 147.64s/it]      


Review pending...

Active Reading Progress:  72%|███████▏  | 0.7216/1 [04:36<01:46, 383.63s/it] 


### 2. Let moderator AI generate questions using the chat endpoint

In [4]:
moderator_thread = superpowered.create_chat_thread(
    knowledge_base_ids=[TOPIC_1_KB_ID, TOPIC_2_KB_ID],
    model='gpt-4',
    system_message= f'You are a moderator for a debate of {topic_1_name} vs. loop {topic_2_name}. '
                    f'Your goal is to communicate on a level that is understandable to the general public. '
                    f'Given how difficult these topics are, you might need to go to great lengths to simplify the concepts. ',
    response_length='short',
)
# generate some questions given the 2 previous document reviews
response = superpowered.get_chat_response(
    thread_id=moderator_thread['id'],
    input=  f'You have been given the following detailed and advanced questions regarding {topic_1_name} and {topic_2_name}: \n\n'
            f'{topic_1_name.upper()} QUESTIONS:\n{topic_1_review}\n\n'
            f'{topic_2_name.upper()} QUESTIONS:\n{topic_2_review}\n\n'
            f'Given the previous questions, please generate 5 questions that get at the heart of the differences between {topic_1_name} and {topic_2_name}.\n\n'
            f'Please respond with a question on each line without number your questions.',
)
questions = response['interaction']['model_response']['content'].split('\n')
for question in questions:
    print(question)

How does Loop Quantum Gravity's background independence contrast with String Theory's background dependence?
In what ways does Loop Quantum Gravity's approach to quantum gravity differ from String Theory's use of extra dimensions and supersymmetry?
What are the key differences in how String Theory and Loop Quantum Gravity attempt to reconcile quantum mechanics with general relativity?
How does the absence of supersymmetry in Loop Quantum Gravity's framework affect its predictions compared to those of String Theory?
What are the challenges Loop Quantum Gravity faces in producing testable predictions, and how do these compare to the challenges faced by String Theory?


### 3. Opening Statements

First, create the chat threads for each AI in the debate and then have them make their opening statements.

In [5]:
topic_1_thread = superpowered.create_chat_thread(
    knowledge_base_ids=[TOPIC_1_KB_ID, TOPIC_2_KB_ID],
    model='gpt-4',
    system_message= f'You are a debater for {topic_1_name}.\n\n'
                    f'Your opponent is debating for {topic_2_name}.\n\n'
                    f'You are convinced {topic_1_name} is the correct {debate_description} and you must communicate this as simply as possible to the general public.',
    response_length='short',
)

topic_2_thread = superpowered.create_chat_thread(
    knowledge_base_ids=[TOPIC_1_KB_ID, TOPIC_2_KB_ID],
    model='gpt-4',
    system_message= f'You are a debater for {topic_2_name}.\n\n'
                    f'Your opponent is debating for {topic_1_name}.\n\n'
                    f'You are convinced {topic_2_name} is the correct {debate_description} and you must communicate this as simply as possible to the general public.',
    response_length='short',
)

In [6]:
opening_statements = f'### {topic_1_name.capitalize()} Opening Statement\n\n'

debate_display = display(Markdown(opening_statements), display_id=True)
topic_1_opening_statement = superpowered.get_chat_response(
    thread_id=topic_1_thread['id'],
    input=f'Please give us an overview of why {topic_1_name} is the optimal {debate_description} and why {topic_2_name} is wrong.',
)['interaction']['model_response']['content']
opening_statements += topic_1_opening_statement + '\n\n'
debate_display.update(Markdown(opening_statements))

opening_statements += f'\n\n### {topic_2_name.capitalize()} Opening Statement\n\n'
topic_2_opening_statement = superpowered.get_chat_response(
    thread_id=topic_2_thread['id'],
    input=f'Please give us an overview of why {topic_2_name} is the optimal {debate_description} and why {topic_1_name} is wrong.',
)['interaction']['model_response']['content']
opening_statements += topic_2_opening_statement + '\n\n'
debate_display.update(Markdown(opening_statements))

### String theory Opening Statement

String Theory is a leading candidate for a Theory of Everything because it unifies all fundamental forces and types of matter within a single framework by modeling particles as one-dimensional strings, and it has shown consistency with established physics like general relativity and quantum field theory in certain limits. Loop Quantum Gravity, while valuable, is less favored because it does not incorporate matter as naturally, lacks experimental evidence for its unique predictions, and has struggled to reproduce established physics in the same comprehensive manner as String Theory.



### Loop quantum gravity Opening Statement

Loop Quantum Gravity (LQG) is a promising Theory of Everything because it is background independent, meaning it doesn't assume a pre-existing spacetime, and it naturally incorporates the quantization of space itself, avoiding the need for unobserved concepts like extra dimensions and supersymmetry that String Theory relies on. Moreover, LQG is formulated in the familiar four dimensions of spacetime, aligning more closely with our observed universe, whereas String Theory requires additional dimensions that have yet to be experimentally confirmed.



### 4. Answer questions from moderator

For each question, start with one thread or the other and then let the conversation develop for a few iterations

In [7]:
debate_output = ''
debate_display = display(Markdown(debate_output), display_id=True)
for question in questions:
    debate_output += f'# {question}\n\n'
    debate_display.update(Markdown(debate_output))
    # GET INITIAL ANSWERS
    st_resp_1 = superpowered.get_chat_response(
        thread_id=topic_1_thread['id'],
        input=question,
    )['interaction']['model_response']['content']
    debate_output += f'### {topic_1_name.capitalize()} (Answer)\n\n{st_resp_1}\n\n'
    debate_display.update(Markdown(debate_output))
    lqg_resp_1 = superpowered.get_chat_response(
        thread_id=topic_2_thread['id'],
        input=question,
    )['interaction']['model_response']['content']
    debate_output += f'### {topic_2_name.capitalize()} (Answer)\n\n{lqg_resp_1}\n\n'
    debate_display.update(Markdown(debate_output))
    # REBUTTALS
    st_resp_2 = superpowered.get_chat_response(
        thread_id=topic_1_thread['id'],
        input=f'Your opponent said: {lqg_resp_1}. Please respond by either pointing out flaws in their argument or by providing a counter argument.',
    )['interaction']['model_response']['content']
    debate_output += f'### {topic_1_name.capitalize()} (Rebuttal)\n\n{st_resp_2}\n\n'
    debate_display.update(Markdown(debate_output))
    lqg_resp_2 = superpowered.get_chat_response(
        thread_id=topic_2_thread['id'],
        input=f'Your opponent said: {st_resp_1}. Please respond by either pointing out flaws in their argument or by providing a counter argument.',
    )['interaction']['model_response']['content']
    debate_output += f'### {topic_2_name.capitalize()} (Rebuttal)\n\n{lqg_resp_2}\n\n'
    debate_display.update(Markdown(debate_output))


# How does Loop Quantum Gravity's background independence contrast with String Theory's background dependence?

### String theory (Answer)

Loop Quantum Gravity is formulated to be background independent, meaning it does not assume a pre-existing spacetime but rather aims to explain the emergence of space and time from its quantum states, aligning with the principles of general relativity. In contrast, String Theory typically requires a fixed spacetime background to define string dynamics, which has raised concerns about its compatibility with the background independence of general relativity, although efforts continue to reconcile this within String Theory, such as through the AdS/CFT correspondence.

### Loop quantum gravity (Answer)

Loop Quantum Gravity is inherently background independent, meaning it does not assume a fixed spacetime geometry; instead, spacetime itself emerges from the quantum properties of loops, making it more aligned with the principles of general relativity. In contrast, String Theory typically requires a pre-defined spacetime background to describe string dynamics, which is seen as a limitation because it does not fully integrate the dynamical nature of spacetime as suggested by general relativity.

### String theory (Rebuttal)

While it's true that Loop Quantum Gravity is background independent and that String Theory has been criticized for its background dependence, this does not necessarily limit String Theory's effectiveness. String theorists argue that the theory is background-independent in a deeper, physical sense, even if the mathematical formulation currently requires a fixed background. Moreover, the AdS/CFT correspondence in String Theory provides a powerful example of how background independence might be realized, suggesting that spacetime can emerge from the theory in a holographic way. This indicates that String Theory has the potential to be fully compatible with the dynamical nature of spacetime, and research is ongoing to develop a more suitable mathematical language to describe this aspect.

### Loop quantum gravity (Rebuttal)

While the AdS/CFT correspondence in String Theory offers a glimpse into background independence, it is limited to specific conditions and does not yet provide a general solution applicable to our universe's observed spacetime. Loop Quantum Gravity, on the other hand, was developed from the ground up to be background independent, offering a framework that does not rely on special conditions and is more directly connected to the dynamic nature of spacetime as described by general relativity, thus providing a more straightforward approach to quantum gravity.

# In what ways does Loop Quantum Gravity's approach to quantum gravity differ from String Theory's use of extra dimensions and supersymmetry?

### String theory (Answer)

Loop Quantum Gravity approaches quantum gravity by quantizing spacetime itself into discrete chunks without invoking extra dimensions or supersymmetry, focusing on the geometry of spacetime at the Planck scale. In contrast, String Theory posits that particles are one-dimensional strings vibrating in a spacetime that includes extra dimensions beyond the familiar four, and often relies on supersymmetry to pair fermions and bosons, which is a symmetry not required by Loop Quantum Gravity. These extra dimensions in String Theory are crucial for its mathematical consistency, while Loop Quantum Gravity remains strictly within the four-dimensional spacetime and does not necessitate additional dimensions or supersymmetric partners.

### Loop quantum gravity (Answer)

Loop Quantum Gravity approaches quantum gravity by quantizing spacetime itself into discrete chunks without invoking extra dimensions or supersymmetry, focusing on the geometry of spacetime at the Planck scale using spin networks and spin foams. In contrast, String Theory posits that fundamental particles are one-dimensional strings vibrating in a spacetime that includes extra dimensions and often relies on supersymmetry to connect different types of particles, which are concepts not yet supported by experimental evidence.

### String theory (Rebuttal)

While it's true that Loop Quantum Gravity does not require extra dimensions or supersymmetry and that these concepts in String Theory lack direct experimental evidence, the strength of String Theory lies in its ability to provide a unified framework that incorporates all known interactions, including gravity, in a consistent quantum theory. The extra dimensions and supersymmetry are theoretical predictions that could resolve many open questions in physics, such as the hierarchy problem and the nature of dark matter. Furthermore, the lack of experimental evidence is not a definitive argument against String Theory, as the energy scales at which these predictions might be directly observable are currently beyond the reach of our experiments. String Theory also has led to many mathematical advances and has provided tools like the AdS/CFT correspondence, which has found applications in various areas of physics, indicating its utility beyond the unconfirmed predictions.

### Loop quantum gravity (Rebuttal)

The reliance of String Theory on extra dimensions and supersymmetry, which lack empirical evidence, can be seen as speculative and a departure from the observable universe. Loop Quantum Gravity's strict adherence to four-dimensional spacetime is not a limitation but a strength, as it seeks to describe quantum gravity in a framework consistent with what we observe, without the need for unverified complexities, thus offering a more parsimonious and potentially verifiable approach to integrating quantum mechanics with general relativity.

# What are the key differences in how String Theory and Loop Quantum Gravity attempt to reconcile quantum mechanics with general relativity?

### String theory (Answer)

String Theory attempts to reconcile quantum mechanics with general relativity by positing that fundamental particles are one-dimensional strings whose vibrations correspond to different particles, potentially unifying all forces and matter in a higher-dimensional framework that includes gravity naturally. Loop Quantum Gravity, on the other hand, seeks to quantize space itself, proposing that space is made of finite loops woven into a network, using a non-perturbative approach that does not require additional dimensions or particles beyond those known, aiming to describe gravity at the quantum level while preserving the background independence of general relativity.

### Loop quantum gravity (Answer)

String Theory attempts to reconcile quantum mechanics with general relativity by positing that fundamental particles are strings vibrating in multiple dimensions, potentially unifying all forces, while Loop Quantum Gravity does so by quantizing the very fabric of spacetime into finite loops within our four-dimensional spacetime, avoiding extra dimensions and focusing solely on the quantum aspects of gravity.

### String theory (Rebuttal)

The description of Loop Quantum Gravity is accurate in that it avoids extra dimensions and focuses on quantizing spacetime itself. However, the inclusion of extra dimensions in String Theory is not a flaw but a feature that allows for a rich structure capable of unifying all fundamental forces, including gravity, within a single theoretical framework. These extra dimensions offer potential explanations for various phenomena in particle physics and cosmology. Additionally, String Theory's reliance on higher dimensions and supersymmetry has led to valuable mathematical insights and tools like the AdS/CFT correspondence, which have applications across physics, indicating a broader utility of the theory beyond its original scope.

### Loop quantum gravity (Rebuttal)

The elegance of String Theory's unification comes at the cost of introducing unobserved entities like extra dimensions, which complicates its predictive power and experimental verification. Loop Quantum Gravity's non-perturbative, background-independent approach offers a more direct and potentially testable route to quantum gravity, focusing on known dimensions and particles, and thus may provide a more immediate connection to our observable universe without the need for additional, unverified theoretical constructs.

# How does the absence of supersymmetry in Loop Quantum Gravity's framework affect its predictions compared to those of String Theory?

### String theory (Answer)

The absence of supersymmetry in Loop Quantum Gravity means it does not predict superpartners for known particles, which simplifies its framework and aligns with the current lack of experimental evidence for supersymmetry. However, this also means it may not offer solutions to certain theoretical issues that supersymmetry addresses in String Theory, such as the hierarchy problem or providing a natural candidate for dark matter. String Theory's use of supersymmetry helps to stabilize the hierarchy between the weak scale and the Planck scale and leads to a rich spectrum of particles that could be detected in future experiments, potentially offering a more comprehensive framework for physics beyond the Standard Model.

### Loop quantum gravity (Answer)

The absence of supersymmetry in Loop Quantum Gravity means it does not predict a host of superpartner particles, unlike String Theory, which requires supersymmetry for stability and consistency. This makes LQG's predictions more directly testable with current experimental technology, as it does not rely on detecting new particles that have yet to be observed, thus potentially offering a more immediate empirical route to validating its framework.

### String theory (Rebuttal)

While it's true that Loop Quantum Gravity's lack of reliance on supersymmetry could be seen as an advantage in terms of testability with current technology, this view overlooks the broader explanatory power of String Theory. Supersymmetry in String Theory not only provides potential stability for the hierarchy of scales but also offers a rich framework that could explain dark matter and unify all fundamental forces, including gravity. The fact that superpartners have not yet been observed does not invalidate String Theory; it may simply indicate that we need to reach higher energy scales to detect them. Moreover, String Theory has led to profound insights in both mathematics and physics, such as the AdS/CFT correspondence, which have applications beyond the original scope of the theory and can be tested in various indirect ways.

### Loop quantum gravity (Rebuttal)

While supersymmetry offers potential solutions to issues like the hierarchy problem, its lack of experimental support after extensive searches casts doubt on its necessity. Loop Quantum Gravity's approach, which does not require supersymmetry, avoids these speculative elements and focuses on explaining gravity at the quantum level with the particles and forces we currently observe. This makes LQG a more conservative and potentially more falsifiable theory, as it does not depend on yet-to-be-discovered physics to resolve theoretical challenges.

# What are the challenges Loop Quantum Gravity faces in producing testable predictions, and how do these compare to the challenges faced by String Theory?

### String theory (Answer)

Loop Quantum Gravity faces challenges in producing testable predictions mainly due to the difficulty in connecting its quantum-scale effects with observable phenomena at larger scales, and the lack of a clear mechanism to incorporate the Standard Model of particle physics. String Theory, while also facing the challenge of connecting high-energy scale predictions to accessible experimental tests, additionally grapples with a vast landscape of possible solutions, making it hard to single out one that corresponds to our universe. Both theories operate at energy scales that are currently beyond the reach of our most powerful experiments, making direct tests challenging for both frameworks.

### Loop quantum gravity (Answer)

Loop Quantum Gravity faces challenges in producing testable predictions mainly due to the difficulty in connecting its Planck-scale phenomena with observable, low-energy scales; however, it is actively developing in areas like cosmology and black hole physics. String Theory also struggles with testability, as its predictions often require energies far beyond current experimental capabilities and the vast "landscape" of solutions complicates making specific predictions. Both theories are pushing the boundaries of physics, but their predictions remain difficult to test with current technology.

### String theory (Rebuttal)

The argument fairly represents the challenges faced by both theories. However, it's worth noting that String Theory's rich mathematical structure has led to tools like the AdS/CFT correspondence, which provides indirect but powerful ways to test aspects of the theory through its implications in fields like nuclear and condensed matter physics. This correspondence has already led to testable predictions, such as the viscosity to entropy ratio in quark-gluon plasmas, which have been confirmed by experiments. While direct tests of String Theory at high energies are currently out of reach, its indirect predictions and the mathematical insights it provides continue to influence and advance theoretical and experimental physics.

### Loop quantum gravity (Rebuttal)

The argument correctly identifies the challenges both theories face in making testable predictions. However, it's worth noting that Loop Quantum Gravity's focus on quantum spacetime and its avoidance of unobserved entities like extra dimensions may offer a more straightforward path to formulating predictions that could be tested at energy scales closer to our current experimental reach, such as through the indirect effects on cosmology and black hole physics, compared to the more complex landscape of String Theory.



### 5. Create a knowledge base with the output of the debate thus far

In [8]:
debate_kb = superpowered.create_knowledge_base(
    title=f'{topic_1_name.capitalize()} vs. {topic_2_name.capitalize()} Debate',
    description=f'A debate between {topic_1_name} and {topic_2_name} on the {debate_description}.',
)

# add opening statements to the debate knowledge base
open_st_1_doc = superpowered.create_document_via_text(
    knowledge_base_id=debate_kb['id'],
    title=f'{topic_1_name.capitalize()} Opening Statement',
    content=topic_1_opening_statement,
)
open_st_2_doc = superpowered.create_document_via_text(
    knowledge_base_id=debate_kb['id'],
    title=f'{topic_2_name.capitalize()} Opening Statement',
    content=topic_2_opening_statement,
)

# add debate output to the debate knowledge base
debate_doc = superpowered.create_document_via_text(
    knowledge_base_id=debate_kb['id'],
    title='Debate Output',
    content=debate_output,
)

# wait for vectorization to complete
while all([open_st_1_doc['vectorization_status'] != 'COMPLETE', open_st_2_doc['vectorization_status'] != 'COMPLETE', debate_doc['vectorization_status'] != 'COMPLETE']):
    open_st_1_doc = superpowered.get_document(debate_kb['id'], open_st_1_doc['id'])
    open_st_2_doc = superpowered.get_document(debate_kb['id'], open_st_2_doc['id'])
    debate_doc = superpowered.get_document(debate_kb['id'], debate_doc['id'])

### 6. Run a document review on the debate knowledge base

In [9]:
active_reading_instructions = f"""\
You are reading through a knowledge base with information regarding a debate between {topic_1_name} and {topic_2_name}.

Please make note of information that you find was more clear from {topic_1_name} or from {topic_2_name}.

Weigh the merits of the arguments made by each side and make note of the evidence and implications of the claims made in the knowledge base.
"""

final_review_instructions = """\
Please give an overall summary of what was discussed. Explain the topics as simply as possible.
"""

# RUN REVIEW FOR DEBATE
debate_review = run_review(debate_kb['id'], active_reading_instructions, final_review_instructions)

The debate between String Theory and Loop Quantum Gravity (LQG) centers on the quest to reconcile quantum mechanics with general relativity into a coherent theory of quantum gravity. The extracted information provides insights into the strengths and weaknesses of both theories, as well as the challenges they face in terms of empirical verification.

Loop Quantum Gravity is praised for its background independence, which aligns with the principles of general relativity. Unlike String Theory, LQG does not require the introduction of extra dimensions or the concept of supersymmetry, both of which lack direct empirical evidence. LQG's approach is more conservative, sticking to the four-dimensional spacetime that we observe and quantizing space itself. This parsimony is seen as a strength because it avoids the introduction of unverified complexities and focuses on a potentially verifiable path to integrating quantum mechanics with general relativity.

On the other hand, String Theory posits that fundamental particles are one-dimensional strings and requires additional dimensions for its mathematical consistency. It also relies on supersymmetry, a symmetry that predicts a partner particle for every particle in the Standard Model. While these features of String Theory address certain theoretical issues, such as the hierarchy problem and the nature of dark matter, they have not been empirically validated. Efforts like the AdS/CFT correspondence aim to reconcile the background dependence of String Theory with the background independence of general relativity, but the theory remains speculative in the absence of experimental evidence.

Both theories face significant challenges in making testable predictions. LQG struggles with connecting Planck-scale phenomena to observable, low-energy scales, although it is making progress in areas like cosmology and black hole physics. String Theory's predictions often require energy levels that are currently beyond experimental reach, and the vast number of possible solutions within the theory makes it difficult to derive specific predictions.

In conclusion, while both String Theory and Loop Quantum Gravity offer intriguing approaches to understanding the fundamental nature of the universe, their predictions are hard to test with today's technology. LQG is commended for its adherence to observable dimensions and a more direct approach, while String Theory provides a broader framework that could potentially address a wider range of theoretical issues. The debate continues as both theories evolve and seek ways to connect their deep theoretical insights with empirical evidence.

Active Reading Progress: 100%|██████████| 1/1 [00:47<00:00, 47.70s/it]


### 8. Generate a long form output based on the debate knowledge base

In [10]:
def long_form_to_md(long_form_response: str) -> str:
    s = ''
    s += '# ' + long_form_response['title'] + '\n\n'
    for section in long_form_response['sections']:
        s += '## ' + section['title'] + '\n'
        s += section['content'] + '\n\n'

    return s


url = f'{SP_BASE_URL}/long_form'
payload = {
    'knowledge_base_ids': [debate_kb['id'], TOPIC_1_KB_ID, TOPIC_2_KB_ID],
    'model': 'mistral-medium',
    'response_length': 'short',
    'prompt':   f'A debate between {topic_1_name} and {topic_2_name} has just taken place and this is the final result: {debate_review}.\n\n'
                f'Please take these results and write a short article discussing the mertics of the arguments made by each side.\n\n'
                f'These are difficult topics so please make sure to simplify the concepts as much as possible.',
}
article_display = display(Markdown('Article pending...'), display_id=True)
resp = requests.post(url, json=payload, auth=API_AUTH)
while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
    resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
    article_md = long_form_to_md(resp.json()['response'])
    article_display.update(Markdown(article_md))

long_form_job_id = resp.json()['id']


# "Debate Between String Theory and Loop Quantum Gravity: Strengths, Challenges, and Future Directions"

## Introduction and Overview
String Theory and Loop Quantum Gravity (LQG) are two prominent approaches in the quest to reconcile quantum mechanics with general relativity, a pursuit often referred to as the search for a theory of quantum gravity. Both theories aim to unify these two fundamental pillars of modern physics, which describe the behavior of matter and energy at vastly different scales.

String Theory posits that fundamental particles are one-dimensional strings vibrating in a higher-dimensional space-time. This theory has the potential to incorporate all known interactions, including gravity, in a consistent quantum framework. However, it requires the existence of extra dimensions and relies on the concept of supersymmetry, which predicts a partner particle for every particle in the Standard Model. These features, while addressing certain theoretical issues, lack direct empirical evidence.

On the other hand, Loop Quantum Gravity focuses on quantizing space itself, adhering strictly to the four-dimensional spacetime that we observe. This approach avoids the introduction of unverified complexities like extra dimensions and supersymmetry, making it more conservative and potentially verifiable. LQG is praised for its background independence, a feature that aligns with the principles of general relativity and distinguishes it from String Theory's background dependence.

Despite their differences, both theories face significant challenges in making testable predictions. LQG struggles to connect phenomena at the Planck scale with observable, low-energy scales, while String Theory's predictions often require energy levels beyond current experimental reach. Additionally, the vast number of possible solutions within String Theory makes it difficult to derive specific predictions.

In summary, String Theory and Loop Quantum Gravity offer unique perspectives on the fundamental nature of the universe. While String Theory provides a broader framework that could potentially address a wider range of theoretical issues, LQG is commended for its adherence to observable dimensions and a more direct approach. The ongoing debate between these two theories highlights the complexity of reconciling quantum mechanics with general relativity and the need for continued research in the field of quantum gravity.

## Loop Quantum Gravity's Case
Loop Quantum Gravity (LQG) offers a unique approach to quantum gravity by focusing on the quantization of spacetime itself, without invoking extra dimensions or supersymmetry. One of the key strengths of LQG is its background independence, which aligns it more closely with the principles of general relativity. Unlike String Theory, LQG does not assume a fixed spacetime background but rather explains the emergence of space and time from its quantum states. This distinction allows LQG to more directly address the dynamical nature of spacetime as suggested by general relativity.

Another advantage of LQG is its strict adherence to four-dimensional spacetime, which is consistent with what we observe in the universe. By avoiding the introduction of unverified complexities like extra dimensions and supersymmetry, LQG presents a more parsimonious and potentially verifiable approach to integrating quantum mechanics with general relativity. This conservative approach focuses on describing quantum gravity using the particles and forces that are already known, rather than relying on yet-to-be-discovered physics to resolve theoretical challenges.

Loop Quantum Gravity has made significant progress in areas like cosmology and black hole physics, despite the challenges it faces in connecting Planck-scale phenomena with observable, low-energy scales. For example, LQG has been used to develop models of quantum cosmology, which aim to explain the origins and evolution of the universe at the quantum level. Additionally, LQG has provided insights into the behavior of black holes, such as the possibility of resolving the information loss paradox through the concept of quantum gravitational phenomena occurring at the event horizon.

In summary, Loop Quantum Gravity's background independence, adherence to four-dimensional spacetime, and avoidance of unverified complexities make it an attractive candidate for a theory of quantum gravity. Its progress in areas like cosmology and black hole physics demonstrates the potential for LQG to provide a more straightforward path to reconciling quantum mechanics with general relativity, compared to the more complex landscape of String Theory.

## String Theory's Case
String Theory's Case: The Allure of a Unified Framework

String Theory's primary strength lies in its ability to provide a unified framework that incorporates all known interactions, including gravity, in a consistent quantum theory. This potential unification is achieved through the concept of one-dimensional strings vibrating in a higher-dimensional space-time, which offers a more comprehensive perspective on the fundamental nature of the universe compared to Loop Quantum Gravity's focus on quantizing space itself.

One significant advantage of String Theory is its potential to address theoretical issues such as the hierarchy problem and the nature of dark matter. The hierarchy problem refers to the large discrepancy between the expected mass of the Higgs boson and its observed mass. String Theory's inclusion of extra dimensions and supersymmetry offers a possible solution to this issue by predicting the existence of partner particles that could help stabilize the Higgs boson's mass. Additionally, the rich spectrum of particles predicted by String Theory could provide a natural candidate for dark matter, an elusive substance that is believed to make up approximately 27% of the universe.

The AdS/CFT correspondence, a tool derived from String Theory, plays a crucial role in reconciling background dependence with background independence. While String Theory is inherently background-dependent, the AdS/CFT correspondence offers a glimpse into background independence by establishing a connection between String Theory in an anti-de Sitter (AdS) space and a conformal field theory (CFT) residing on the boundary of that space. This correspondence has led to many mathematical advances and has found applications in various areas of physics, indicating its utility beyond unconfirmed predictions.

Despite its strengths, String Theory faces challenges, particularly the lack of empirical evidence for extra dimensions and supersymmetry. The energy scales at which these predictions might be directly observable are currently beyond the reach of our experiments. However, the absence of experimental evidence is not a definitive argument against String Theory, as the search for new particles and phenomena at higher energy scales continues. The ongoing development of String Theory and its potential to address a wide range of theoretical issues make it an intriguing and valuable approach in the quest for a theory of quantum gravity.

## Challenges and Future Directions
Both Loop Quantum Gravity and String Theory face significant challenges in making testable predictions and connecting their theoretical insights with empirical evidence. For Loop Quantum Gravity, the primary challenge lies in connecting its quantum-scale effects with observable phenomena at larger scales. This is due to the vast difference in energy scales between the Planck scale, where quantum gravity effects become significant, and the scales accessible to current experiments. Additionally, incorporating the Standard Model of particle physics into the LQG framework remains an open research question.

String Theory, on the other hand, grapples with the challenge of connecting its high-energy scale predictions to accessible experimental tests. Furthermore, the vast landscape of possible solutions within the theory makes it difficult to single out one that corresponds to our universe. Both theories operate at energy scales that are currently beyond the reach of our most powerful experiments, making direct tests challenging.

Despite these challenges, ongoing efforts are being made to overcome these obstacles and find ways to test the predictions of both theories. For Loop Quantum Gravity, researchers are actively developing its applications in areas like cosmology and black hole physics, where indirect effects on observable phenomena may provide opportunities for testing its predictions. In String Theory, the rich mathematical structure has led to tools like the AdS/CFT correspondence, which offers indirect but powerful ways to test aspects of the theory through its implications in fields like nuclear and condensed matter physics. This correspondence has already led to testable predictions, such as the viscosity to entropy ratio in quark-gluon plasmas, which have been confirmed by experiments.

In conclusion, while both Loop Quantum Gravity and String Theory face significant challenges in making testable predictions and connecting their theoretical insights with empirical evidence, ongoing research and the development of new tools and techniques offer hope for future advancements in the field of quantum gravity. The quest to unify quantum mechanics with general relativity remains a central focus of modern physics, and continued progress in this area has the potential to revolutionize our understanding of the fundamental nature of the universe.



### 8. Export article to audio

In [11]:
url = f'{SP_BASE_URL}/long_form/{long_form_job_id}/exports/audio'
payload = {
    'audio_voice_name': 'openai_alloy',
    'link_ttl_minutes': 24 * 60,
}
resp = requests.post(url, json=payload, auth=API_AUTH)
while resp.json()['status'] not in {'COMPLETE', 'FAILED'}:
    time.sleep(2)
    resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
mp3_url = resp.json()['response']['download_url']

print(mp3_url)

https://files.superpowered.ai/long_form_exports/074fef7d-39fe-4239-a333-eb696e42667a/2ffc8bdc-7d03-4b63-ac7a-bfea8c65c210/quantum_gravity_debate_v0__openai_alloy.mp3?Expires=1708069643&Signature=V24PFB2JJus2unVhqOisM0JC1-feZ3~OAQWfM-03m9uEdowIUYQ2hV28UT9m5eWCzR7gQ5683E6NMIyl7ruwqGZBa0jPw1Am8~4LAFCngPrdjbrNjQwo2zeL8rnNzJILAW0ZYFgvNXlQjq~J~zHEiOS1j3UOmknsr~-~yryQO~lGq45kE8Q8hZ~bjw0IihfA8sMK4VKzSUjvVZ~fW~MuTJ-UgGDljSJm7nze3ZkV9yBgZfKwFsCU6iwlcPYEZQ9URva-8k-WJHK9c1HVqEKhlXgTy4rz5s~4kZ8VofziRlCGzAisv17PUi3mrk420JHMc2sXEpHi0hLYEA24WxhS2A__&Key-Pair-Id=KN9MQLKJLYE2H
